# Comparation & evaluation of recommender systems 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pickle
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from numpy import mean,sqrt,square
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import auxiliar_functions as af
from datetime import timedelta

In [ ]:
variables_interes=['pyrite law', 'chalcopyrite law',
       'chalcocite law', 'covelin law', 'crusher index', 'sag power index',
       'ball work index', 'bornite law','charge cell', 'speed','HH TPH', 'granulometry','Edad','loss of TPH',"solid percentage","HH charge cell","LL charge cell","TPH",
       #"recommendation base model carl","recommendation base model acn",
       "water","power"]

In [ ]:
# read df full csv
df_features=pd.read_csv('../../data/processed data/df_feature_engineering_18December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')[variables_interes]
df_features.dropna(inplace=True)
df_features.sort_index(inplace=True)
df_features.shape

In [ ]:
# Test
df_features=pd.concat([df_features["2020-04"],df_features["2021-09"],df_features["2022"]])
df_features.shape

# Recommendation base model ACN

In [ ]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//models//pickles//Modelo update Accenture//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//models//pickles//Modelo update Accenture//cc_recommendations_prod.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK['s'+subcontexto].loc[granulometria]["cc"]
  
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [ ]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [ ]:
# Recomendación 
Rec_HH_CC=df_features.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_features["recommendation base model acn"]=Rec_HH_CC

# Recommendation base model Carl

In [ ]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//models//pickles//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//models//pickles//Curvas de celda de carga vs granulometria productivo actualizado.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK[subcontexto].loc[granulometria]["cc"]
    
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [ ]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [ ]:
# Recomendación 
Rec_HH_CC=df_features.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_features["recommendation base model carl"]=Rec_HH_CC

# Recommendation ML model

In [ ]:
df_rec_ml_model=pd.read_csv('../../data/processed data/recommendation_30March2023.csv',parse_dates=["Timestamp"],index_col="Timestamp").rename(columns={"start loss tph":"predict start loss tph"}).drop(columns=["loss of TPH"])
print(df_rec_ml_model.shape)
df_rec_ml_model.head()

In [ ]:
df_merge=df_features.merge(df_rec_ml_model, left_index=True, right_index=True,how='left',suffixes=('_x', '_y'))
df_merge.head()

# Distribution recommendations

In [ ]:
df_merge[["recommendation base model acn","recommendation base model carl","recommended HH charge cell"]].describe()

In [ ]:

fig = go.Figure()
fig.add_trace(go.Histogram(x=df_merge["recommended HH charge cell"],name="recommendation ML model",opacity=1))
fig.add_trace(go.Histogram(x=df_merge["recommendation base model carl"],name="recommendation base model carl",opacity=0.75))
fig.add_trace(go.Histogram(x=df_merge["recommendation base model acn"],name="recommendation base model acn",opacity=0.5))

fig.update_layout(barmode='stack')
fig.update_layout(height=600, width=1200, title_text="Distribution of recommendations")
fig.show()

# Visualization

In [ ]:
titulos=tuple(["TPH & HH TPH","charge cell","granulometry & SPI","speed",'water',"solid percentage","start loss tph","loss of TPH"])
fig = make_subplots(
    rows=8, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["TPH optimum"],
                    mode='lines',
                    name='TPH predict opt'),row=1, col=1)

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["TPH predict"],
                    mode='lines',
                    name='TPH predict'),row=1, col=1)
fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["HH TPH"],
                    mode='lines',
                    name='HH TPH'),row=1, col=1)

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["TPH"],
                    mode='lines',
                    name='TPH'),row=1, col=1)

##
fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["HH charge cell"],
                    mode='lines',
                    name='HH charge cell'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge['recommendation base model acn'],
                    mode='lines',
                    name='recommendation base model acn'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge['recommendation base model carl'],
                    mode='lines',
                    name='recommendation base model carl'),row=2, col=1)


fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["recommended HH charge cell"],
                    mode='lines',
                    name='recommended HH charge cell ML model'),row=2, col=1)


fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["LL charge cell"],
                    mode='lines',
                    name='LL charge cell'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["charge cell"],
                    mode='lines',
                    name='charge cell'),row=2, col=1)  


fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,)),row=3, col=1)  


fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, )),row=3, col=1)  


fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, )),row=4, col=1)  

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, )),row=5, col=1)  

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge['water'], ##FF6511
                    mode='lines',
                    name='water',line=dict(width=3, )),row=6, col=1)  


fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["predict start loss tph"],
                    mode='lines',
                    name="start loss TPH"),row=7, col=1)

fig.add_trace(go.Scatter(x=df_merge.index, y=df_merge["loss of TPH"],
                    mode='lines',
                    name="loss of TPH"),row=8, col=1)


fig.update_layout(height=2000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")


fig.show()

# Evaluation

In [ ]:
sistem_recommender='recommendation base model carl'

def metric_system_recommendation(sistem_recommender,df1):

    df1["dif_TPH_HH"]=df1["HH TPH"]-df1["TPH"]
    df1["dif_CC_HH"]=df1["HH charge cell"]-df1["charge cell"]
    df1["dif_CC_LL"]=df1["charge cell"]-df1["LL charge cell"]

    # Información de perdida de TPH en los proximos 5 minutos
    for i in range(1,6):
        df1[f"loss of TPH_{i}"]=df1["loss of TPH"].shift(i)

    df1["greater than HH CC"]=df1["dif_CC_HH"].apply(lambda x: 0 if x>1 else 1)
    for i in range(1,6):
        df1[f"greater than HH CC_{i}"]=df1["greater than HH CC"].shift(i)

    df1["less than LL CC"]=df1["dif_CC_LL"].apply(lambda x: 0 if x>1 else 1)
    for i in range(1,6):
        df1[f"less than LL CC_{i}"]=df1["less than LL CC"].shift(i)

    df1["greater than Rec HH CC"]=df1.apply(lambda x: 1 if x[sistem_recommender]<=x["charge cell"] else 0,axis=1)
    for i in range(1,6):
        df1[f"greater than Rec HH CC_{i}"]=df1["greater than Rec HH CC"].shift(i)

    df1["Rec HH CC less than HH CC"]=df1.apply(lambda x: 1 if x[sistem_recommender]<x["HH charge cell"] else 0,axis=1)
    for i in range(1,6):
        df1[f"Rec HH CC less than HH CC_{i}"]=df1["Rec HH CC less than HH CC"].shift(i)

    df1.dropna(inplace=True)
    df1["loss of TPH"]=df1.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or (x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)
    df1=af.loss_tph(df1,"loss of TPH")

    df_full=df1.copy()

    list_df=[]
    for i in df_full[df_full["start loss tph"]==1].index:
        try:
            #i="2022-02-27 12:31:00"
            inicio_perdida=i
            test=df_full[inicio_perdida:inicio_perdida+timedelta(hours=5)]
            fin_perdida=min(test[(test["end loss tph"]==1)].index)+timedelta(minutes=1)
            inicio_evento=inicio_perdida-timedelta(minutes=5)
            data=df_full.copy()[inicio_evento:fin_perdida]
            #display(data)
            dic_df={"inicio evento":inicio_evento,"inicio perdida":inicio_perdida,"fin perdida":fin_perdida,"variación HH TPH":data["HH TPH"].std()!=0,"Actua sistema de control":any(data["greater than HH CC"]==1),
            "Supera recomendación del modelo":any(data["greater than Rec HH CC"]==1),"Recomendación menor a HH CC":any(data["Rec HH CC less than HH CC"]==1),"CC menor a LL CC":any(data["less than LL CC"]==1),
            "Causalidad":any(data[data["greater than Rec HH CC"]==1].index<=inicio_perdida)}
            list_df.append(pd.DataFrame(dic_df,index=[1]))  
        except:
            #print(i)
            pass

    df_tph_loss=pd.concat(list_df).reset_index(drop=True)

    # Perdida de TPH por superar recomendación [+]
    evento1=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==True) \
        & (df_tph_loss["Supera recomendación del modelo"]==True)\
        & (df_tph_loss["Causalidad"]==True)].reset_index(drop=True)#.sample(1)
    list_loss_tph=[]
    for i in range(len(evento1)):
        data=df_full[evento1.iloc[i]["inicio perdida"]:evento1.iloc[i]["fin perdida"]]
        list_loss_tph.append(data["dif_TPH_HH"].sum())
    tph1=sum(list_loss_tph)

    # Perdida de TPH sin superar recomendación [-]
    evento2=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==False)\
        & (df_tph_loss["Supera recomendación del modelo"]==False)\
        ].reset_index(drop=True)
    list_loss_tph=[]
    for i in range(len(evento2)):
        data=df_full[evento2.iloc[i]["inicio perdida"]:evento2.iloc[i]["fin perdida"]]
        list_loss_tph.append(data["dif_TPH_HH"].sum())
    tph2=sum(list_loss_tph)
    print(tph1)
    print(tph2)
    metric_final=tph1-tph2
    
    return metric_final


In [ ]:
sistem_recommender='recommendation base model carl'
metric_system_recommendation(sistem_recommender,df_merge)

In [ ]:
sistem_recommender="recommendation base model acn"
metric_system_recommendation(sistem_recommender,df_merge)

In [ ]:
sistem_recommender="recommended HH charge cell"
metric_system_recommendation(sistem_recommender,df_merge)